In [ ]:
import logging
import pandas as pd
from docx import Document
from docx.shared import Inches, Pt, RGBColor
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
from datetime import datetime
from bs4 import BeautifulSoup
from datetime import datetime
import requests




# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# HighBond API configuration
API_TOKEN = "e6589a07a91e1604d711c78ef1b8c091fc7c09119f11d2c4aad948fc53942675"
BASE_URL = "https://apis-eu.highbond.com/v1/orgs/48414"
HEADERS = {
    "Authorization": f"Bearer {API_TOKEN}",
    "Content-Type": "application/vnd.api+json"
}

def get_all_projects():
    url = f"{BASE_URL}/projects"
    response = requests.get(url, headers=HEADERS)
    if response.status_code == 200:
        return response.json()['data']
    else:
        logger.error(f"Failed to get projects: {response.status_code} - {response.text}")
        return []

def get_project_issues(project_id):
    url = f"{BASE_URL}/projects/{project_id}/issues"
    response = requests.get(url, headers=HEADERS)
    if response.status_code == 200:
        return response.json()['data']
    else:
        logger.error(f"Failed to get issues for project {project_id}: {response.status_code} - {response.text}")
        return []

def clean_html(value):
    if isinstance(value, (int, float)):
        return str(value)
    if not isinstance(value, str):
        return str(value)

    soup = BeautifulSoup(value, 'html.parser')

    # Check if there is a <table> tag and if so, convert it to a Word table
    if soup.find('table'):
        return convert_html_table_to_word(soup.find('table'))
    
    for script in soup(["script", "style"]):
        script.decompose()
    for br in soup.find_all("br"):
        br.replace_with("\n")
    text = soup.get_text()
    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = '\n'.join(chunk for chunk in chunks if chunk)
    return text

def convert_html_table_to_word(table_soup):
    """Converts an HTML table into a Word table."""
    table = []
    rows = table_soup.find_all('tr')

    for row in rows:
        cells = row.find_all(['td', 'th'])
        table.append([cell.get_text(strip=True) for cell in cells])
    
    word_table = ""
    for row in table:
        word_table += "| " + " | ".join(row) + " |\n"

    return word_table

def prompt_filters():
    region_filter = input("Enter region filter (partial match allowed): ")
    month_filter = input("Enter month filter (YYYY-MM): ")
    return region_filter, month_filter

def create_word_report(table_data, headers, region_filter, month_filter):
    doc = Document()

    # -------- Define your styles --------
    normal_style = doc.styles['Normal']
    normal_style.font.name = 'Calibri'
    normal_style.font.size = Pt(11)

    h1_style = doc.styles['Heading 1']
    h1_style.font.name = 'Calibri'
    h1_style.font.size = Pt(16)
    h1_style.font.color.rgb = RGBColor.from_string('107AB8')

    h2_style = doc.styles['Heading 2']
    h2_style.font.name = 'Calibri'
    h2_style.font.size = Pt(12)
    h2_style.font.color.rgb = RGBColor.from_string('EF6149')

    # -------- Cover Page (with logos) --------
    # This remains in Section 1
    header = doc.sections[0].header
    header_table = header.add_table(rows=1, cols=2)
    header_table.autofit = True
    c0, c1 = header_table.rows[0].cells
    try:
        c0.paragraphs[0].add_run().add_picture('minigroup_logo.png', width=Inches(1.5))
        p = c1.paragraphs[0]
        p.alignment = WD_ALIGN_PARAGRAPH.RIGHT
        p.add_run().add_picture('eleven_degrees_logo.png', width=Inches(1.5))
    except Exception as e:
        logging.warning(f"Could not load logos: {e}")

    title = doc.add_paragraph()
    title_run = title.add_run("Regional Issues Report")
    title_run.font.name = 'Calibri'
    title_run.font.size = Pt(24)
    title_run.font.bold = True
    title_run.font.color.rgb = RGBColor.from_string('107AB8')
    title.alignment = WD_ALIGN_PARAGRAPH.CENTER

    doc.add_paragraph(f"Mini Group / Eleven Degrees Consulting", style='Normal').alignment = WD_ALIGN_PARAGRAPH.CENTER
    doc.add_paragraph(f"Date: {datetime.today().strftime('%Y-%m-%d')}", style='Normal').alignment = WD_ALIGN_PARAGRAPH.CENTER
    doc.add_page_break()

    # -------- Table of Contents --------
    toc = doc.add_paragraph()
    fld_char_begin = OxmlElement('w:fldChar'); fld_char_begin.set(qn('w:fldCharType'), 'begin')
    instr_text = OxmlElement('w:instrText'); instr_text.set(qn('xml:space'), 'preserve')
    instr_text.text = 'TOC \\o "1-3" \\h \\z \\u'
    fld_char_separate = OxmlElement('w:fldChar'); fld_char_separate.set(qn('w:fldCharType'), 'separate')
    text = OxmlElement('w:t'); text.text = "Right-click to update Table of Contents"
    fld_char_separate.append(text)
    fld_char_end = OxmlElement('w:fldChar'); fld_char_end.set(qn('w:fldCharType'), 'end')
    r = toc.runs[0]._r
    r.append(fld_char_begin); r.append(instr_text); r.append(fld_char_separate); r.append(fld_char_end)

    doc.add_page_break()

    # -------- Loop through projects --------
    current_project = None
    first_project = True

    for row in table_data:
        project_id, project_name = row[0], row[1]
        branch, region, start_date, status = row[2], row[3], row[4], row[5]
        bm, om, sup = row[14], row[15], row[16]

        # New section (and page) for each project except the very first
        if not first_project:
            section = doc.add_section(WD_SECTION.NEW_PAGE)
            # inject header in this new section:
            hdr = section.header
            hdr_para = hdr.paragraphs[0] if hdr.paragraphs else hdr.add_paragraph()
            hdr_para.alignment = WD_ALIGN_PARAGRAPH.CENTER
            hdr_para.font.size = Pt(9)
            hdr_para.font.name = 'Calibri'
            hdr_para.text = (
                f"Branch: {branch}    |    Region: {region}    |    "
                f"Start Date: {start_date}    |    Status: {status}    |    "
                f"Branch Manager: {bm}    |    Operations Manager: {om}    |    Supervisor: {sup}"
            )
        else:
            # For the very first project, reuse Section 1 but still add a header with details:
            hdr = doc.sections[0].header
            hdr_para = hdr.add_paragraph()
            hdr_para.alignment = WD_ALIGN_PARAGRAPH.CENTER
            hdr_para.font.size = Pt(9)
            hdr_para.font.name = 'Calibri'
            hdr_para.text = (
                f"Branch: {branch}    |    Region: {region}    |    "
                f"Start Date: {start_date}    |    Status: {status}    |    "
                f"Branch Manager: {bm}    |    Operations Manager: {om}    |    Supervisor: {sup}"
            )
            first_project = False

        # Project heading
        doc.add_heading(f"Project: {project_name}", level=1)

        # Loop through issues for this row
        doc.add_heading(f"Issue: {row[6]}", level=2)
        tbl = doc.add_table(rows=0, cols=2)
        tbl.style = 'Light List Accent 1'
        def add_row(label, value):
            cells = tbl.add_row().cells
            cells[0].text = label
            cells[1].text = clean_html(value)

        add_row("Severity", row[7])
        add_row("Description", row[8])
        add_row("Implication", row[9])
        add_row("Cost Impact", f"${row[10]}")
        add_row("Management Comment 1", row[11])
        add_row("Management Comment 2", row[12])
        add_row("Recommendation", row[13])

        doc.add_paragraph()

    return doc

def main():
    try:
        logger.info("Starting the script")
        region_filter, month_filter = prompt_filters()
        logger.info(f"Filters: Region - {region_filter}, Month - {month_filter}")

        projects = get_all_projects()
        logger.info(f"Retrieved {len(projects)} projects")

        table_data = []
        headers = [
            "Project ID", "Project Name", "Branch", "Region", "Start Date", "Status",
            "Issue Title", "Severity", "Description", "Implication", "Cost Impact",
            "Management Comments 1", "Management Comments 2", "Recommendation",
            "Branch Manager", "Operations Manager", "Supervisor"
        ]

        for project in projects:
            try:
                project_id = project.get("id")
                if not project_id:
                    logger.warning(f"Project ID not found: {project}")
                    continue

                project_attributes = project.get("attributes", {})
                project_name = project_attributes.get("name", "N/A")
                start_date = project_attributes.get("start_date", "")
                status = project_attributes.get("status", "N/A")
                project_custom_fields = project_attributes.get("custom_attributes", [])

                region = next((f.get("value", "N/A") for f in project_custom_fields if f.get("term") == "Custom field 2"), "N/A")
                branch = next((f.get("value", "N/A") for f in project_custom_fields if f.get("term") == "Custom field 8"), "N/A")
                branch_manager = next((f.get("value", "N/A") for f in project_custom_fields if f.get("term") == "Custom field 5"), "N/A")
                operations_manager = next((f.get("value", "N/A") for f in project_custom_fields if f.get("term") == "Custom field 6"), "N/A")
                supervisor = next((f.get("value", "N/A") for f in project_custom_fields if f.get("term") == "Custom field 7"), "N/A")

                if region_filter and region_filter.lower() not in region.lower():
                    logger.info(f"Skipping project {project_name} due to region filter")
                    continue
                if month_filter and not start_date.startswith(month_filter):
                    logger.info(f"Skipping project {project_name} due to month filter")
                    continue

                issues = get_project_issues(project_id)
                if len(issues) == 0:
                    continue

                for issue in issues:
                    try:
                        attributes = issue.get("attributes", {})
                        title = attributes.get("title", "")
                        severity = attributes.get("severity", "")
                        description = clean_html(attributes.get("description", ""))
                        implication = clean_html(attributes.get("effect", ""))
                        cost_impact = str(attributes.get("cost_impact", 0))
                        recommendation = clean_html(attributes.get("recommendation", ""))

                        issue_custom_attributes = attributes.get("custom_attributes", [])
                        mgmt_comment_1 = next((field.get("value", "") for field in issue_custom_attributes if field.get("term") == "Custom field 1"), "")
                        mgmt_comment_2 = next((field.get("value", "") for field in issue_custom_attributes if field.get("term") == "Custom field 2"), "")

                        row = [
                            project_id, project_name, branch, region, start_date, status,
                            title, severity, description, implication, cost_impact,
                            mgmt_comment_1, mgmt_comment_2, recommendation,
                            branch_manager, operations_manager, supervisor
                        ]
                        table_data.append(row)
                    except Exception as e:
                        logger.error(f"Error processing issue for project {project_name}: {str(e)}")

            except Exception as e:
                logger.error(f"Error processing project {project.get('id', 'Unknown')}: {str(e)}")

        logger.info(f"Total issue rows prepared: {len(table_data)}")

        if len(table_data) == 0:
            logger.warning("No matching issues found")
        else:
            df = pd.DataFrame(table_data, columns=headers)
            df.to_csv("project_data.csv", index=False)
            logger.info("Data saved to 'project_data.csv'")

            doc = create_word_report(table_data, headers, region_filter, month_filter)
            doc.save("project_report.docx",)
            logger.info("Report generated and saved as 'project_report.docx'")

    except Exception as e:
        logger.error(f"An error occurred in the main function: {str(e)}")
        logger.exception("Exception details:")

if __name__ == "__main__":
    main()

2025-06-20 10:51:35,823 - INFO - Starting the script
2025-06-20 10:51:37,768 - INFO - Filters: Region - , Month - 
2025-06-20 10:51:43,560 - INFO - Retrieved 50 projects
2025-06-20 10:53:12,442 - ERROR - Error processing project 128292: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
2025-06-20 10:53:38,991 - INFO - Total issue rows prepared: 368
2025-06-20 10:53:39,019 - INFO - Data saved to 'project_data.csv'
2025-06-20 10:53:39,044 - ERROR - An error occurred in the main function: BlockItemContainer.add_table() missing 1 required positional argument: 'width'
2025-06-20 10:53:39,045 - ERROR - Exception details:
Traceback (most recent call last):
  File "/tmp/ipykernel_9515/4000105020.py", line 290, in main
    doc = create_word_report(table_data, headers, region_filter, month_filter)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_9515/4000105020.py", line 109, in create_word_report
    header_table 

In [2]:
import sys
!{sys.executable} -m pip install python-docx


In [3]:
import logging
import pandas as pd

from docx import Document
from docx.shared import Inches, Pt, RGBColor
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.enum.section import WD_ORIENT
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
from datetime import datetime
from bs4 import BeautifulSoup
import requests

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# HighBond API configuration
API_TOKEN = "e6589a07a91e1604d711c78ef1b8c091fc7c09119f11d2c4aad948fc53942675"
BASE_URL = "https://apis-eu.highbond.com/v1/orgs/48414"
HEADERS = {
    "Authorization": f"Bearer {API_TOKEN}",
    "Content-Type": "application/vnd.api+json"
}

def get_all_projects():
    url = f"{BASE_URL}/projects"
    response = requests.get(url, headers=HEADERS)
    if response.status_code == 200:
        return response.json()['data']
    else:
        logger.error(f"Failed to get projects: {response.status_code} - {response.text}")
        return []

def get_project_issues(project_id):
    url = f"{BASE_URL}/projects/{project_id}/issues"
    response = requests.get(url, headers=HEADERS)
    if response.status_code == 200:
        return response.json()['data']
    else:
        logger.error(f"Failed to get issues for project {project_id}: {response.status_code} - {response.text}")
        return []

def clean_html(value):
    if isinstance(value, (int, float)):
        return str(value)
    if not isinstance(value, str):
        return str(value)
    soup = BeautifulSoup(value, 'html.parser')
    if soup.find('table'):
        return convert_html_table_to_word(soup.find('table'))
    for tag in soup(["script", "style"]): tag.decompose()
    for br in soup.find_all("br"): br.replace_with("\n")
    text = soup.get_text()
    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    return '\n'.join(chunk for chunk in chunks if chunk)

def convert_html_table_to_word(table_soup):
    table = []
    for row in table_soup.find_all('tr'):
        cells = row.find_all(['td', 'th'])
        table.append([cell.get_text(strip=True) for cell in cells])
    word_table = ""
    for row in table:
        word_table += "| " + " | ".join(row) + " |\n"
    return word_table

def prompt_filters():
    region_filter = input("Enter region filter (partial match allowed): ")
    month_filter = input("Enter month filter (YYYY-MM): ")
    return region_filter, month_filter

def add_background_footer(section):
    footer = section.footer
    paragraph = footer.paragraphs[0]
    run = paragraph.add_run()
    run.text = " " * 200  # force footer area height
    shading_elm = OxmlElement('w:shd')
    shading_elm.set(qn('w:fill'), '107AB8')  # background color
    shading_elm.set(qn('w:val'), 'clear')
    run._r.get_or_add_rPr().append(shading_elm)

def create_word_report(table_data, headers, region_filter, month_filter):
    doc = Document()

    # Set landscape orientation and margins
    section = doc.sections[0]
    section.orientation = WD_ORIENT.LANDSCAPE
    section.page_width, section.page_height = Inches(11), Inches(8.5)
    for s in doc.sections:
        s.top_margin = s.bottom_margin = Inches(0.5)
        s.left_margin = s.right_margin = Inches(0.5)

    # Define styles
    styles = doc.styles
    styles['Normal'].font.name = 'Calibri'
    styles['Normal'].font.size = Pt(11)

    styles['Heading 1'].font.size = Pt(16)
    styles['Heading 1'].font.color.rgb = RGBColor.from_string('107AB8')

    styles['Heading 2'].font.size = Pt(13)
    styles['Heading 2'].font.color.rgb = RGBColor.from_string('EF6149')

    # Cover Page
    doc.add_paragraph().add_run("Regional Issues Report").bold = True
    doc.paragraphs[-1].alignment = WD_ALIGN_PARAGRAPH.CENTER
    doc.paragraphs[-1].runs[0].font.size = Pt(24)
    doc.paragraphs[-1].runs[0].font.color.rgb = RGBColor.from_string('107AB8')

    subtitle = doc.add_paragraph("Mini Group / Eleven Degrees Consulting")
    subtitle.alignment = WD_ALIGN_PARAGRAPH.CENTER
    subtitle.runs[0].font.size = Pt(14)

    doc.add_paragraph(f"Date: {datetime.today().strftime('%Y-%m-%d')}").alignment = WD_ALIGN_PARAGRAPH.CENTER
    add_background_footer(doc.sections[0])
    doc.add_page_break()

    # Table of Contents
    toc = doc.add_paragraph()
    run = toc.add_run()
    fldChar = OxmlElement('w:fldChar'); fldChar.set(qn('w:fldCharType'), 'begin')
    instrText = OxmlElement('w:instrText'); instrText.text = 'TOC \\o "1-2" \\h \\z \\u'
    fldChar2 = OxmlElement('w:fldChar'); fldChar2.set(qn('w:fldCharType'), 'separate')
    fldChar3 = OxmlElement('w:fldChar'); fldChar3.set(qn('w:fldCharType'), 'end')
    r_element = run._r
    r_element.append(fldChar)
    r_element.append(instrText)
    r_element.append(fldChar2)
    r_element.append(fldChar3)
    doc.add_page_break()

    # Group data by project
    from collections import defaultdict
    grouped_data = defaultdict(list)
    for row in table_data:
        if row[7].lower() in ['high', 'medium']:
            grouped_data[row[0]].append(row)

    for pid, rows in grouped_data.items():
        project_name = rows[0][1]
        branch, region, start_date, status = rows[0][2], rows[0][3], rows[0][4], rows[0][5]
        bm, om, sup = rows[0][14], rows[0][15], rows[0][16]

        # -- Page Header per project --
        section = doc.add_section(start_type=1)
        header = section.header
        header_table = header.add_table(rows=2, cols=3, width=Inches(9))
        header_table.autofit = True
        header_table.alignment = WD_ALIGN_PARAGRAPH.CENTER

        try:
            header_table.cell(0, 0).paragraphs[0].add_run().add_picture("minigroup_logo.png", width=Inches(1.2))
            header_table.cell(0, 2).paragraphs[0].add_run().add_picture("eleven_degrees.jpg", width=Inches(1.2))
        except Exception as e:
            logger.warning(f"Header logo loading failed: {e}")

        header_text = header_table.cell(1, 1).paragraphs[0]
        header_text.alignment = WD_ALIGN_PARAGRAPH.CENTER
        run = header_text.add_run(f"Branch: {branch} | Region: {region} | Start: {start_date} | Status: {status}\n"
                                  f"BM: {bm} | OM: {om} | Sup: {sup}")
        run.font.size = Pt(9)

        doc.add_heading(f"Project: {project_name}", level=1)

        for row in rows:
            doc.add_heading(f"Issue: {row[6]}", level=2)
            table = doc.add_table(rows=0, cols=2)
            table.style = 'Light List Accent 1'

            def add_row(label, value):
                row_cells = table.add_row().cells
                row_cells[0].text = label
                row_cells[1].text = clean_html(value)

            add_row("Severity", row[7])
            add_row("Description", row[8])
            add_row("Implication", row[9])
            add_row("Cost Impact", f"${row[10]}")
            add_row("Management Comment 1", row[11])
            add_row("Management Comment 2", row[12])
            add_row("Recommendation", row[13])
            doc.add_paragraph()

    # Custom Footer with Styled Page Numbers
    for section in doc.sections:
        footer = section.footer
        para = footer.paragraphs[0]
        para.alignment = WD_ALIGN_PARAGRAPH.RIGHT
        run = para.add_run()
        fldChar = OxmlElement('w:fldChar'); fldChar.set(qn('w:fldCharType'), 'begin')
        instrText = OxmlElement('w:instrText'); instrText.text = 'PAGE'
        fldChar2 = OxmlElement('w:fldChar'); fldChar2.set(qn('w:fldCharType'), 'separate')
        fldChar3 = OxmlElement('w:fldChar'); fldChar3.set(qn('w:fldCharType'), 'end')
        r_element = run._r
        r_element.append(fldChar)
        r_element.append(instrText)
        r_element.append(fldChar2)
        r_element.append(fldChar3)
        run.font.size = Pt(9)
        run.font.color.rgb = RGBColor.from_string('808080')

    return doc

def main():
    try:
        logger.info("Starting script")
        region_filter, month_filter = prompt_filters()
        projects = get_all_projects()
        table_data = []
        headers = [
            "Project ID", "Project Name", "Branch", "Region", "Start Date", "Status",
            "Issue Title", "Severity", "Description", "Implication", "Cost Impact",
            "Management Comments 1", "Management Comments 2", "Recommendation",
            "Branch Manager", "Operations Manager", "Supervisor"
        ]

        for project in projects:
            try:
                pid = project.get("id")
                if not pid:
                    continue
                attr = project.get("attributes", {})
                name = attr.get("name", "N/A")
                start = attr.get("start_date", "")
                status = attr.get("status", "N/A")
                custom = attr.get("custom_attributes", [])

                region = next((f.get("value", "N/A") for f in custom if f.get("term") == "Custom field 2"), "N/A")
                branch = next((f.get("value", "N/A") for f in custom if f.get("term") == "Custom field 8"), "N/A")
                bm = next((f.get("value", "N/A") for f in custom if f.get("term") == "Custom field 5"), "N/A")
                om = next((f.get("value", "N/A") for f in custom if f.get("term") == "Custom field 6"), "N/A")
                sup = next((f.get("value", "N/A") for f in custom if f.get("term") == "Custom field 7"), "N/A")

                if region_filter and region_filter.lower() not in region.lower(): continue
                if month_filter and not start.startswith(month_filter): continue

                issues = get_project_issues(pid)
                if not issues: continue

                for issue in issues:
                    severity = issue["attributes"].get("severity", "").lower()
                    if severity not in ["high", "medium"]:
                        continue
                    i_attr = issue.get("attributes", {})
                    title = i_attr.get("title", "")
                    desc = clean_html(i_attr.get("description", ""))
                    effect = clean_html(i_attr.get("effect", ""))
                    cost = str(i_attr.get("cost_impact", 0))
                    rec = clean_html(i_attr.get("recommendation", ""))
                    icustom = i_attr.get("custom_attributes", [])
                    cm1 = next((f.get("value", "") for f in icustom if f.get("term") == "Custom field 1"), "")
                    cm2 = next((f.get("value", "") for f in icustom if f.get("term") == "Custom field 2"), "")

                    row = [pid, name, branch, region, start, status, title, severity, desc, effect, cost, cm1, cm2, rec, bm, om, sup]
                    table_data.append(row)

            except Exception as e:
                logger.error(f"Error processing project {project.get('id')}: {e}")

        if table_data:
            df = pd.DataFrame(table_data, columns=headers)
            df.to_csv("project_data.csv", index=False)
            logger.info("CSV saved")

            doc = create_word_report(table_data, headers, region_filter, month_filter)
            doc.save("project_report.docx")
            logger.info("Word report saved")

        else:
            logger.warning("No matching issues found")

    except Exception as e:
        logger.error(f"Main error: {e}")
        logger.exception("Traceback:")

if __name__ == "__main__":
    main()


2025-06-16 11:36:52,578 - INFO - Starting script
2025-06-16 11:37:21,917 - WARNING - No matching issues found


In [ ]:
#!/usr/bin/env python3
"""
project_report.py

Generate a Regional Issues Report from HighBond projects,
only for projects starting on or before today, with robust
retry, pagination, and relative-URL handling.
Supports filtering by region, month (YYYY-MM), and issue severity.
"""

import sys
import argparse
import logging
import re
from datetime import datetime, date
from collections import defaultdict
from urllib.parse import urljoin

import requests
import pandas as pd
from bs4 import BeautifulSoup

from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

from docx import Document
from docx.shared import Inches, Pt, RGBColor
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.enum.section import WD_ORIENT
from docx.oxml import OxmlElement
from docx.oxml.ns import qn

# ─── Logger ───────────────────────────────────────────────────
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

# ─── API Constants ──────────────────────────────────────────
API_TOKEN = "acd6c44de072af279f19042267e98f0a70ca00c5966e118636dd87a451786347"
BASE_URL  = "https://apis-eu.highbond.com/v1/orgs/48414"
HEADERS   = {
    "Authorization": f"Bearer {API_TOKEN}",
    "Content-Type": "application/vnd.api+json"
}

# ─── Session with Retry ───────────────────────────────────────
session = requests.Session()
retries = Retry(
    total=5,
    backoff_factor=1,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["GET"]
)
adapter = HTTPAdapter(max_retries=retries)
session.mount("https://", adapter)
session.mount("http://",  adapter)

# ─── HTTP Helpers ────────────────────────────────────────────
def get_all_projects():
    today_str = date.today().isoformat()
    url    = f"{BASE_URL}/projects"
    page   = 1
    params = {
        "filter[start_date][lte]": today_str,
        "page[size]": 100,
        "page[number]": page
    }

    all_projects = []
    while True:
        try:
            resp = session.get(url, headers=HEADERS, params=params, timeout=10)
            resp.raise_for_status()
        except Exception as e:
            logger.error(f"Failed to fetch projects page {page}: {e}")
            break

        body = resp.json()
        batch = body.get("data", [])
        all_projects.extend(batch)

        next_link = body.get("links", {}).get("next")
        if not next_link:
            break

        url = urljoin(BASE_URL, next_link)
        params = None
        page += 1

    filtered = []
    for p in all_projects:
        sd = p.get("attributes", {}).get("start_date", "")
        try:
            if datetime.strptime(sd, "%Y-%m-%d").date() <= date.today():
                filtered.append(p)
        except ValueError:
            continue

    logger.info(f"Retrieved {len(filtered)} projects up to {today_str}")
    return filtered


def get_project_issues(pid):
    try:
        resp = session.get(f"{BASE_URL}/projects/{pid}/issues", headers=HEADERS, timeout=10)
        resp.raise_for_status()
    except Exception as e:
        logger.error(f"Failed to fetch issues for project {pid}: {e}")
        return []
    return resp.json().get("data", [])

# ─── HTML Cleaning & Table Extraction ────────────────────────
def clean_html_and_extract_tables(value):
    if not isinstance(value, str):
        return str(value), []
    soup = BeautifulSoup(value, "html.parser")
    for tag in soup(["script", "style"]): tag.decompose()
    for br in soup.find_all("br"): br.replace_with("\n")

    tables = []
    for table in soup.find_all("table"):
        headers = [th.get_text(strip=True) for th in table.find_all("th")]
        rows    = []
        for tr in table.find_all("tr"):
            cells = [td.get_text(strip=True) for td in tr.find_all("td")]
            if any(cells): rows.append(cells)
        if headers or rows: tables.append((headers, rows))
        table.decompose()

    return soup.get_text().strip(), tables

# ─── Column Width Calculation ─────────────────────────────────
def _compute_column_widths(text_matrix, max_total_width_inches=10.0, min_width_inches=0.5):
    if not text_matrix: return []
    cols = list(zip(*text_matrix))
    scores, total = [], 0.0
    for col in cols:
        lengths = [len(str(c)) for c in col]
        mx = max(lengths)
        has_sent = any(len(str(c).split())>5 for c in col)
        is_num   = all(str(c).replace(".","",1).isdigit() for c in col if c)
        weight = 1.5 if has_sent else 0.5 if is_num else 1.0
        scores.append(mx*weight); total+=mx*weight
    total = total or 1.0
    widths = [(s/total)*max_total_width_inches for s in scores]
    widths = [max(min_width_inches,w) for w in widths]
    used = sum(widths)
    if used>max_total_width_inches:
        factor = max_total_width_inches/used
        widths = [w*factor for w in widths]
    return [Inches(w) for w in widths]

# ─── Mini-Table Insertion ────────────────────────────────────
def add_mini_table_to_cell(cell, headers, rows):
    header_len = len(headers) if headers else 0
    row_lens    = [len(r) for r in rows]
    lengths     = [header_len] + row_lens
    col_count   = max(lengths) if lengths else 0

    matrix = []
    if headers:
        matrix.append([headers[i] if i<header_len else "" for i in range(col_count)])
    for r in rows:
        matrix.append([r[i] if i<len(r) else "" for i in range(col_count)])

    max_w = getattr(cell, "width", Inches(5)).inches
    col_w = _compute_column_widths(matrix, max_total_width_inches=max_w)

    mini = cell.add_table(rows=1 if headers else 0, cols=col_count)
    mini.style, mini.autofit = "Light Grid Accent 1", False

    if headers:
        for i, txt in enumerate(matrix[0]):
            c = mini.rows[0].cells[i]; c.width=col_w[i]
            p = c.paragraphs[0]; r=p.add_run(txt); r.bold=True
            p.paragraph_format.space_before=p.paragraph_format.space_after=Pt(0)

    for row in matrix[1 if headers else 0:]:
        rc = mini.add_row().cells
        for i, txt in enumerate(row):
            rc[i].width=col_w[i]
            p=rc[i].paragraphs[0]; p.text=txt
            p.paragraph_format.space_before=p.paragraph_format.space_after=Pt(0)
            p.paragraph_format.line_spacing=1.0

# ─── Page Numbers Footer ─────────────────────────────────────
def add_global_page_numbers(doc):
    for sec in doc.sections:
        ftr=sec.footer; ftr.is_linked_to_previous=False
        p=ftr.paragraphs[0]; p.alignment=WD_ALIGN_PARAGRAPH.RIGHT
        run=p.add_run()
        for fld_type in ("begin","separate","end"):
            fld=OxmlElement("w:fldChar"); fld.set(qn("w:fldCharType"),fld_type)
            run._r.append(fld)
        instr=OxmlElement("w:instrText"); instr.text="PAGE"
        run._r.insert(1,instr)
        run.font.size=Pt(9); run.font.color.rgb=RGBColor(128,128,128)

# ─── Build the DOCX Report ───────────────────────────────────
def create_word_report(table_data):
    doc = Document()
    sec=doc.sections[0]
    sec.orientation,sec.page_width,sec.page_height=WD_ORIENT.LANDSCAPE,Inches(11),Inches(8.5)
    for s in doc.sections: s.top_margin=s.bottom_margin=s.left_margin=s.right_margin=Inches(0.5)

    doc.styles["Normal"].font.name,doc.styles["Normal"].font.size="Calibri",Pt(11)
    doc.styles["Heading 1"].font.size=Pt(16); doc.styles["Heading 1"].font.color.rgb=RGBColor.from_string("107AB8")
    doc.styles["Heading 2"].font.size=Pt(13); doc.styles["Heading 2"].font.color.rgb=RGBColor.from_string("EF6149")

    # Cover
    p=doc.add_paragraph(); p.alignment=WD_ALIGN_PARAGRAPH.CENTER
    r=p.add_run("Regional Issues Report\n"); r.font.size, r.font.color.rgb=Pt(24),RGBColor.from_string("107AB8")
    doc.add_paragraph("Mini Group / Eleven Degrees Consulting").alignment=WD_ALIGN_PARAGRAPH.CENTER
    doc.add_paragraph(f"Date: {datetime.today():%Y-%m-%d}").alignment=WD_ALIGN_PARAGRAPH.CENTER
    doc.add_page_break()

    add_global_page_numbers(doc)
    grouped=defaultdict(list)
    for row in table_data: grouped[row[0]].append(row)
    usable=Inches(10)

    for pid,rows in grouped.items():
        proj=rows[0]
        doc.add_heading(f"Project: {proj[1]}",level=1)
        hdr=doc.add_paragraph()
        for label,idx in [("Branch",2),("Region",3),("Start Date",4),("Status",5),("Branch Manager",14),("Operations Manager",15),("Supervisor",16)]:
            run=hdr.add_run(f"{label}: {proj[idx]}\n"); run.bold=True
        doc.add_paragraph()

        for r in rows:
            clean_vals={}
            for key,idx in [("Description",8),("Implication",9),("Management Comment 1",11),("Management Comment 2",12),("Recommendation",13)]:
                txt,tbls=clean_html_and_extract_tables(r[idx]); clean_vals[key]=(txt,tbls)
            cost_val=r[10] if isinstance(r[10],(int,float)) else 0.0
            fields=[("Issue Title",(r[6],[])),("Severity",(r[7],[])),("Description",clean_vals["Description"]),("Implication",clean_vals["Implication"]),("Cost Impact",(f"${cost_val:,.2f}",[])),("Management Comment 1",clean_vals["Management Comment 1"]),("Management Comment 2",clean_vals["Management Comment 2"]),("Recommendation",clean_vals["Recommendation"])]
            matrix=[[lbl,val[0]] for lbl,val in fields]; col_widths=_compute_column_widths(matrix,max_total_width_inches=usable.inches)
            tbl=doc.add_table(rows=len(fields),cols=2); tbl.style,tbl.autofit="Table Grid",False; tbl.columns[0].width,tbl.columns[1].width=col_widths
            for i,(lbl,(txt,mini)) in enumerate(fields): left,right=tbl.rows[i].cells; left.text=lbl; left.paragraphs[0].runs[0].bold=True; right.text=txt; [add_mini_table_to_cell(right,h,rw) for h,rw in mini]
            doc.add_paragraph()
        doc.add_page_break()
    return doc

# ─── Entrypoint ─────────────────────────────────────────────
def main():
    parser=argparse.ArgumentParser(description="Generate Regional Issues Report from HighBond projects")
    parser.add_argument("--region", help="Partial, case-insensitive filter on region")
    parser.add_argument("--month", help="Start-date filter in YYYY-MM (optional)")
    parser.add_argument("--severity", choices=["high","medium","low"], help="Filter issues by severity")
    args,_=parser.parse_known_args()

    region_filter=(args.region or input("► Region filter (partial, Enter to skip): ")).strip().lower()
    month_filter=args.month or input("► Month filter (YYYY-MM, Enter to skip): ").strip()
    severity_filter=args.severity or input("► Severity filter (high/medium/low, Enter for all): ")).strip().lower()

    if month_filter:
        try:
            datetime.strptime(month_filter, "%Y-%m")
        except ValueError:
            logger.error("❌ Invalid month format. Use YYYY-MM (e.g. 2025-06).")
            sys.exit(1)
    valid_sevs={"high","medium","low"}
    if severity_filter and severity_filter not in valid_sevs:
        logger.error(f"❌ Invalid severity. Choose from {valid_sevs}.")
        sys.exit(1)

    severities = {severity_filter} if severity_filter else valid_sevs

    table_data=[]
    projects=get_all_projects()
    projects.sort(key=lambda p: p.get("attributes",{}).get("start_date",""),reverse=True)

    for pr in projects:
        start=pr.get("attributes",{}).get("start_date","")
        try:
            if datetime.strptime(start, "%Y-%m-%d").date()>date.today(): continue
        except ValueError:
            continue

        pid=pr.get("id"); attr=pr.get("attributes",{}); name=attr.get("name","{}"); status=attr.get("status","")
        ca=attr.get("custom_attributes",[])
        # pull and normalize
        def normalize(v):
            if isinstance(v,list): return ", ".join(str(x).strip() for x in v)
            return str(v)
        region=normalize(next((c["value"] for c in ca if c.get("term")=="Region"),[]))
        branch=normalize(next((c["value"] for c in ca if c.get("term")=="Branch"),[]))
        bm=normalize(next((c["value"] for c in ca if c.get("term")=="Branch Manager"),[]))
        om=normalize(next((c["value"] for c in ca if c.get("term")=="Operations Manager"),[]))
        sup=normalize(next((c["value"] for c in ca if c.get("term")=="Supervisor"),[]))

        if region_filter and region_filter not in region.lower(): continue
        if month_filter and not start.startswith(month_filter): continue

        logger.info(f"Fetching issues for {name!r} (Start={start}, Region={region!r})")
        for isd in get_project_issues(pid):
            ia=isd.get("attributes",{})
            sev=str(ia.get("severity","")).lower()
            if sev not in severities: continue
            cm={c["term"]:c["value"] for c in ia.get("custom_attributes",[])}
            cost=ia.get("cost_impact") if isinstance(ia.get("cost_impact"),(int,float)) else 0.0
            table_data.append([pid,name,branch,region,start,status,ia.get("title",""),sev.capitalize(),ia.get("description",""),ia.get("effect",""),cost,cm.get("Custom field 1",""),cm.get("Region",""),ia.get("recommendation",""),bm,om,sup])

    if not table_data:
        logger.warning("⚠️ No data matched your filters.")
        return

    pd.DataFrame(table_data).to_csv("project_data.csv",index=False)
    safe_region=re.sub(r"\W+","_",region_filter or "ALL")
    safe_month=re.sub(r"\W+","_",month_filter) if month_filter else "ALL"
    fname=f"project_report_{safe_region}_{safe_month}.docx"

    doc=create_word_report(table_data)
    doc.save(fname)
    logger.info(f"✅ Report saved as {fname}")

if __name__=="__main__": main()


SyntaxError: unmatched ')' (375863825.py, line 252)

In [ ]:
#!/usr/bin/env python3
"""
project_report.py

Generate a Regional Issues Report from HighBond projects,
only for projects starting on or before today, with robust
retry, pagination, and relative-URL handling.
"""

import sys
import argparse
import logging
import re
from datetime import datetime, date
from collections import defaultdict
from urllib.parse import urljoin

import requests
import pandas as pd
from bs4 import BeautifulSoup

from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry


import re
import subprocess
from pdf2docx import Converter  


from docx import Document
from docx.shared import Inches, Pt, RGBColor
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.enum.section import WD_ORIENT
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
from docx.enum.table import WD_ROW_HEIGHT_RULE
from docx.enum.section import WD_SECTION


# ─── Logger ───────────────────────────────────────────────────
def ensure_str(val):
    return ", ".join(val) if isinstance(val, list) else str(val or "")

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

# ─── API Constants ────────────────────────────────────────
API_TOKEN = "acd6c44de072af279f19042267e98f0a70ca00c5966e118636dd87a451786347"
BASE_URL  = "https://apis-eu.highbond.com/v1/orgs/48414"
HEADERS   = {
    "Authorization": f"Bearer {API_TOKEN}",
    "Content-Type": "application/vnd.api+json"
}

# ─── Session with Retry ───────────────────────────────────────
session = requests.Session()
retries = Retry(
    total=5,
    backoff_factor=1,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["GET"]
)
adapter = HTTPAdapter(max_retries=retries)
session.mount("https://", adapter)
session.mount("http://",  adapter)

def convert_pdf_to_docx(pdf_path: str, docx_path: str):
    """Convert a PDF file to DOCX using pdf2docx."""
    try:
        cv = Converter(pdf_path)
        cv.convert(docx_path, start=0, end=None)
        cv.close()
        logger.info(f"✅ PDF→DOCX: '{docx_path}'")
    except Exception as e:
        logger.error(f"❌ PDF→DOCX conversion error: {e}")

# ─── HTTP Helpers ────────────────────────────────────────────
def get_all_projects():
    today_str = date.today().isoformat()
    url    = f"{BASE_URL}/projects"
    page   = 1
    params = {
        "filter[start_date][lte]": today_str,
        "page[size]": 100,
        "page[number]": page
    }
    all_projects = []
    while True:
        try:
            resp = session.get(url, headers=HEADERS, params=params, timeout=10)
            resp.raise_for_status()
        except Exception as e:
            logger.error(f"Failed to fetch projects page {page}: {e}")
            break

        body = resp.json()
        all_projects.extend(body.get("data", []))
        next_link = body.get("links", {}).get("next")
        if not next_link:
            break
        url = urljoin(BASE_URL, next_link)
        params = None
        page += 1

    # filter out any that somehow slipped through with future dates
    filtered = []
    for p in all_projects:
        sd = p.get("attributes", {}).get("start_date", "")
        try:
            if datetime.strptime(sd, "%Y-%m-%d").date() <= date.today():
                filtered.append(p)
        except ValueError:
            continue

    logger.info(f"Retrieved {len(filtered)} projects up to {today_str}")
    return filtered

def get_project_issues(pid):
    try:
        resp = session.get(f"{BASE_URL}/projects/{pid}/issues", headers=HEADERS, timeout=10)
        resp.raise_for_status()
    except Exception as e:
        logger.error(f"Failed to fetch issues for project {pid}: {e}")
        return []
    return resp.json().get("data", [])

# ─── HTML Cleaning & Table Extraction ────────────────────────
def clean_html_and_extract_tables(value):
    if not isinstance(value, str):
        return str(value), []
    soup = BeautifulSoup(value, "html.parser")
    for tag in soup(["script", "style"]):
        tag.decompose()
    for br in soup.find_all("br"):
        br.replace_with("\n")

    tables = []
    for table in soup.find_all("table"):
        headers = [th.get_text(strip=True) for th in table.find_all("th")]
        rows    = []
        for tr in table.find_all("tr"):
            cells = [td.get_text(strip=True) for td in tr.find_all("td")]
            if any(cells):
                rows.append(cells)
        if headers or rows:
            tables.append((headers, rows))
        table.decompose()

    return soup.get_text().strip(), tables

# ─── Column Width Calculation ─────────────────────────────────
def _compute_column_widths(text_matrix, max_total_width_inches=10.0, min_width_inches=0.5):
    if not text_matrix:
        return []
    cols = list(zip(*text_matrix))
    scores, total = [], 0.0
    for col in cols:
        lengths = [len(str(c)) for c in col]
        mx = max(lengths)
        has_sent = any(len(str(c).split()) > 5 for c in col)
        is_num   = all(str(c).replace(".", "", 1).isdigit() for c in col if c)
        weight = 1.5 if has_sent else 0.5 if is_num else 1.0
        scores.append(mx * weight)
        total += mx * weight
    total = total or 1.0
    widths = [(s/total)*max_total_width_inches for s in scores]
    widths = [max(min_width_inches, w) for w in widths]
    used = sum(widths)
    if used > max_total_width_inches:
        factor = max_total_width_inches / used
        widths = [w * factor for w in widths]
    return [Inches(w) for w in widths]

# ─── Mini-Table Insertion ────────────────────────────────────
def add_mini_table_to_cell(cell, headers, rows):
    header_len = len(headers) if headers else 0
    row_lens = [len(r) for r in rows]
    col_count = max([header_len] + row_lens or [0])
    matrix = []
    if headers:
        matrix.append([headers[i] if i < header_len else "" for i in range(col_count)])
    for r in rows:
        matrix.append([r[i] if i < len(r) else "" for i in range(col_count)])

    max_w = getattr(cell, "width", Inches(5)).inches
    col_w = _compute_column_widths(matrix, max_total_width_inches=max_w)

    mini = cell.add_table(rows=1 if headers else 0, cols=col_count)
    mini.style, mini.autofit = "Light Grid Accent 1", False

    if headers:
        for i, txt in enumerate(matrix[0]):
            c = mini.rows[0].cells[i]
            c.width = col_w[i]
            p = c.paragraphs[0]
            run = p.add_run(txt); run.bold = True
            p.paragraph_format.space_before = p.paragraph_format.space_after = Pt(0)

    for row in matrix[1 if headers else 0:]:
        rc = mini.add_row().cells
        for i, txt in enumerate(row):
            rc[i].width = col_w[i]
            p = rc[i].paragraphs[0]
            p.text = txt
            p.paragraph_format.space_before = p.paragraph_format.space_after = Pt(0)
            p.paragraph_format.line_spacing = 1.0

# ─── Build the DOCX Report ───────────────────────────────────
def create_word_report(table_data, region_filter):
    doc = Document()

    # ─── Setup Landscape & Margins ─────────────────────────
    sect0 = doc.sections[0]
    sect0.orientation = WD_ORIENT.LANDSCAPE
    sect0.page_width, sect0.page_height = Inches(11), Inches(8.5)
    for sect in doc.sections:
        sect.top_margin = sect.bottom_margin = sect.left_margin = sect.right_margin = Inches(0.5)

    # ─── Styles ─────────────────────────────────────────────
    normal = doc.styles["Normal"]
    normal.font.name, normal.font.size = "Calibri", Pt(11)
    h1 = doc.styles["Heading 1"]
    h1.font.name, h1.font.size, h1.font.color.rgb = "Calibri", Pt(16), RGBColor.from_string("107AB8") 
    h2 = doc.styles["Heading 2"]
    h2.font.name, h2.font.size, h2.font.color.rgb = "Calibri", Pt(13), RGBColor.from_string("EF6149")

    # ─── Cover Page ────────────────────────────────────────
    p = doc.add_paragraph()
    p.alignment = WD_ALIGN_PARAGRAPH.CENTER
    r = p.add_run("Regional Issues Report\n")
    r.font.size, r.font.color.rgb = Pt(24), RGBColor.from_string("107AB8")
    doc.add_paragraph("Mini Group / Eleven Degrees Consulting")\
       .alignment = WD_ALIGN_PARAGRAPH.CENTER
    doc.add_paragraph(f"Date: {datetime.today():%Y-%m-%d}")\
       .alignment = WD_ALIGN_PARAGRAPH.CENTER
    doc.add_page_break()

    # ─── Running Footer ────────────────────────────────────
    for sect in doc.sections:
        ftr = sect.footer
        ftr.is_linked_to_previous = False
        # clear existing paras
        for p in list(ftr.paragraphs):
            ftr._element.remove(p._element)

        # center region text
        p_reg = ftr.add_paragraph(f"Regional Issues Report for “{region_filter or 'ALL'}” region")
        p_reg.alignment = WD_ALIGN_PARAGRAPH.CENTER
        p_reg.runs[0].font.name, p_reg.runs[0].font.size = "Calibri", Pt(9)

        # right page number
        p_pg = ftr.add_paragraph()
        p_pg.alignment = WD_ALIGN_PARAGRAPH.RIGHT
        run_pg = p_pg.add_run()
        for fld in ("begin", "separate", "end"):
            el = OxmlElement("w:fldChar"); el.set(qn("w:fldCharType"), fld)
            run_pg._r.append(el)
        instr = OxmlElement("w:instrText"); instr.text = "PAGE"
        run_pg._r.insert(1, instr)
        run_pg.font.size, run_pg.font.color.rgb = Pt(9), RGBColor(128, 128, 128)

    # ─── Group & Render Projects ───────────────────────────
    grouped = defaultdict(list)
    for row in table_data:
        grouped[row[0]].append(row)

    first = True
    for pid, rows in grouped.items():
        proj = rows[0]
        name, branch, region, start, status = proj[1], proj[2], proj[3], proj[4], proj[5]
        bm, om, sup = proj[14], proj[15], proj[16]

        if not first:
            section = doc.add_section(WD_SECTION.NEW_PAGE)
        else:
            section = doc.sections[0]
            first = False

        # ─── Header Table (2×2) ────────────────────────────
        header = section.header
        header.is_linked_to_previous = False
        for p in list(header.paragraphs):
            header._element.remove(p._element)

        tbl = header.add_table(rows=2, cols=2, width=Inches(11))
        tbl.autofit = False

        # remove borders
        tbl_pr = tbl._tbl.tblPr or OxmlElement("w:tblPr")
        if tbl._tbl.tblPr is None:
            tbl._tbl.append(tbl_pr)
        borders = OxmlElement("w:tblBorders")
        for edge in ("top", "left", "bottom", "right", "insideH", "insideV"):
            b = OxmlElement(f"w:{edge}"); b.set(qn("w:val"), "nil")
            borders.append(b)
        tbl_pr.append(borders)

        # equal widths
        tbl.columns[0].width = tbl.columns[1].width = Inches(5.5)

        # ─ Logos Row ───────────────────────────────────────────
        row0 = tbl.rows[0]
        row0.height, row0.height_rule = Inches(2.0), WD_ROW_HEIGHT_RULE.EXACTLY

        # left logo in col 0
        cell_l = tbl.cell(0, 0)
        cell_l.vertical_alignment = WD_ALIGN_PARAGRAPH.CENTER
        tcPr = cell_l._tc.get_or_add_tcPr()
        tcMar = OxmlElement("w:tcMar")
        left_pad = OxmlElement("w:left"); left_pad.set(qn("w:w"), "200"); left_pad.set(qn("w:type"), "dxa")
        tcMar.append(left_pad); tcPr.append(tcMar)
        p_l = cell_l.paragraphs[0]; p_l.alignment = WD_ALIGN_PARAGRAPH.LEFT
        run_l = p_l.add_run()
        try:
            run_l.add_picture("minigroup.png", width=Inches(1.5))
        except Exception:
            logger.warning("Minigroup logo load failed")

        # right logo in col 1
        cell_r = tbl.cell(0, 1)
        cell_r.vertical_alignment = WD_ALIGN_PARAGRAPH.CENTER
        p_r = cell_r.paragraphs[0]; p_r.alignment = WD_ALIGN_PARAGRAPH.RIGHT
        run_r = p_r.add_run()
        try:
            run_r.add_picture("minigroup_logo.png", width=Inches(1.5))
        except Exception:
            logger.warning("Eleven Degrees logo load failed")

        # ─ Metadata Row ───────────────────────────────────────
        row1 = tbl.rows[1]
        mcell = tbl.cell(1, 0).merge(tbl.cell(1, 1))
        mcell.height, mcell.height_rule = Inches(0.5), WD_ROW_HEIGHT_RULE.EXACTLY
        mcell.vertical_alignment = WD_ALIGN_PARAGRAPH.CENTER
        p_meta = mcell.paragraphs[0]; p_meta.alignment = WD_ALIGN_PARAGRAPH.CENTER
        run_meta = p_meta.add_run(
            f"Branch: {branch}   |   Region: {region}   |   Start: {start}   |   "
            f"Status: {status}   |   BM: {bm}   |   OM: {om}   |   Sup: {sup}"
        )
        run_meta.font.name, run_meta.font.size = "Calibri", Pt(9)

        # ─── Body Content ────────────────────────────────────
        doc.add_heading(f"Project: {name}", level=1)
        for k, v in {
            "Branch":branch, "Region":region, "Start Date":start,
            "Status":status, "Branch Manager":bm,
            "Operations Manager":om, "Supervisor":sup
        }.items():
            p = doc.add_paragraph(); run = p.add_run(f"{k}: "); run.bold = True
            p.add_run(ensure_str(v))
        doc.add_paragraph()

        # ─── Issues ─────────────────────────────────────────
        for issue in rows:
            desc, tables = clean_html_and_extract_tables(issue[8])
            doc.add_heading(f"Issue: {issue[6]}", level=2)

            fields = [
                ("Severity", issue[7]), ("Description", desc),
                ("Implication", issue[9]), ("Cost Impact", f"${issue[10]:,.2f}"),
                ("Mgmt Comment 1", issue[11]), ("Mgmt Comment 2", issue[12]),
                ("Recommendation", issue[13]),
            ]
            tbl_i = doc.add_table(rows=len(fields), cols=2)
            tbl_i.style, tbl_i.autofit = "Table Grid", False
            widths = _compute_column_widths(
                [[lbl, val] for lbl, val in fields],
                max_total_width_inches=Inches(10).inches
            )
            tbl_i.columns[0].width, tbl_i.columns[1].width = widths

            for i, (lbl, val) in enumerate(fields):
                c0, c1 = tbl_i.rows[i].cells
                c0.text = lbl
                c0.paragraphs[0].runs[0].bold = True
                # guard None → empty string
                c1.text = str(val) if val is not None else ""
                for hdrs, rows_tbl in tables:
                    add_mini_table_to_cell(c1, hdrs, rows_tbl)
            doc.add_paragraph()

    return doc

# ─── Entrypoint ─────────────────────────────────────────────
def main():
    parser = argparse.ArgumentParser(description="Generate Regional Issues Report from HighBond projects")
    parser.add_argument("--region", help="Partial, case-insensitive filter on region")
    parser.add_argument("--month",  help="Start-date filter in YYYY-MM (optional)")
    parser.add_argument("--severity", help="Comma-separated severities to include (e.g. High,Medium,Low)")
    args, _ = parser.parse_known_args()

    region_filter = (args.region or input("► Region filter (partial, Enter to skip): ")).strip().lower()
    month_filter  = args.month  or input("► Month filter (YYYY-MM, Enter to skip): ").strip()
    sev_input     = args.severity or input("► Severity filter (comma-separated, e.g. High,Medium. Enter to skip): ")
    severity_filters = {s.strip().lower() for s in sev_input.split(",") if s.strip()} if sev_input else set()

    if month_filter:
        try:
            datetime.strptime(month_filter, "%Y-%m")
        except ValueError:
            logger.error("❌ Invalid month format. Use YYYY-MM (e.g. 2025-06).")
            sys.exit(1)

    table_data = []
    projects = get_all_projects()
    projects.sort(key=lambda x: x.get("attributes", {}).get("start_date", ""), reverse=True)

    for pr in projects:
        start = pr["attributes"].get("start_date", "")
        try:
            if datetime.strptime(start, "%Y-%m-%d").date() > date.today():
                continue
        except ValueError:
            continue

        pid    = pr["id"]
        attr   = pr["attributes"]
        name   = attr.get("name", "")
        status = attr.get("status", "")

        ca     = attr.get("custom_attributes", [])
        region = ensure_str(next((c["value"] for c in ca if c.get("term")=="Region"), ""))
        branch = ensure_str(next((c["value"] for c in ca if c.get("term")=="Branch"), ""))
        bm     = ensure_str(next((c["value"] for c in ca if c.get("term")=="Branch Manager"), ""))
        om     = ensure_str(next((c["value"] for c in ca if c.get("term")=="Operations Manager"), ""))
        sup    = ensure_str(next((c["value"] for c in ca if c.get("term")=="Supervisor"), ""))

        if region_filter and region_filter not in region.lower(): continue
        if month_filter  and not start.startswith(month_filter):   continue

        logger.info(f"Fetching issues for {name!r} (Start={start}, Region={region!r})")
        for isd in get_project_issues(pid):
            ia  = isd.get("attributes", {})
            sev = str(ia.get("severity","")).strip().lower()
            if severity_filters and sev not in severity_filters:
                continue

            cm       = {c["term"]: c["value"] for c in ia.get("custom_attributes",[])}
            cost_val = ia.get("cost_impact") if isinstance(ia.get("cost_impact"), (int,float)) else 0.0

            table_data.append([
                pid, name, branch, region, start, status,
                ia.get("title",""), sev.capitalize(),
                ia.get("description",""), ia.get("effect",""),
                cost_val, cm.get("Custom field 1",""), cm.get("Region",""),
                ia.get("recommendation",""), bm, om, sup
            ])

    if not table_data:
        logger.warning("⚠️ No data matched your filters."); return

    pd.DataFrame(table_data).to_csv("project_data.csv", index=False)

    safe_region = re.sub(r"\W+", "_", region_filter or "ALL")
    safe_month  = re.sub(r"\W+", "_", month_filter or "ALL")
    base        = f"project_report_{safe_region}_{safe_month}"
    docx_fname  = f"{base}.docx"
    pdf_fname   = f"{base}.pdf"
    recon_docx  = f"{base}_reconverted.docx"

    # 1) Generate Word
    doc = create_word_report(table_data, region_filter)
    doc.save(docx_fname)
    logger.info(f"✅ Word saved: {docx_fname}")

    # 2) Word → PDF via LibreOffice
    try:
        subprocess.run(
            ["libreoffice", "--headless", "--convert-to", "pdf", docx_fname],
            check=True
        )
        # LibreOffice names output "<base>.pdf" in fdrseawcwd
        logger.info(f"✅ Word→PDF via LibreOffice: {pdf_fname}")
    except subprocess.CalledProcessError as e:
        logger.error(f"❌ LibreOffice conversion failed: {e}")

    # 3) PDF → Word via pdf2docx
    convert_pdf_to_docx(pdf_fname, recon_docx)


if __name__ == "__main__":
    main()

[INFO] Retrieved 280 projects up to 2025-06-24
[INFO] Fetching issues for 'Mtondia June 2025' (Start=2025-06-17, Region='Mombasa')
[INFO] Fetching issues for 'Majengo 1  June 2025' (Start=2025-06-10, Region='Mombasa')
[INFO] Fetching issues for 'Bamburi 2  June 2025' (Start=2025-06-03, Region='Mombasa')
[INFO] Fetching issues for 'Bamburi 1 June 2025' (Start=2025-06-01, Region='Mombasa')
/tmp/ipykernel_170948/3939032138.py:289: FutureWarning: Truth-testing of elements was a source of confusion and will always return True in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  tbl_pr = tbl._tbl.tblPr or OxmlElement("w:tblPr")
[INFO] ✅ Word saved: project_report_mombasa_2025_06.docx
[INFO] ✅ Word→PDF via LibreOffice: project_report_mombasa_2025_06.pdf
[INFO] Start to convert project_report_mombasa_2025_06.pdf
[INFO] [1/4] Opening document...
[INFO] [2/4] Analyzing document...
[WARNING] 'modified' timestamp seems very low; regarding as unix timestamp
[WARNING] '

In [3]:
import sys
print(sys.executable)
print(sys.path[:3], "…")


/home/kitavidouglas/anaconda3/bin/python
['/home/kitavidouglas/Desktop/DataAnalyst', '/home/kitavidouglas/anaconda3/lib/python312.zip', '/home/kitavidouglas/anaconda3/lib/python3.12'] …
